# 12.2 PyTorch深度学习建模可视化工具TensorBoard的安装与使用

在深度学习建模过程中，为了能够快速绘制模型的基本结构/观察模型评价指标伴随训练过程的动态变化情况，当然也为了能够观察数据图像数据，我们可以使用TensroBoard工具来进行PyTorch深度学习模型可视化展示。

TensorBoard原本是TensorFlow的可视化工具，而目前在TensorboardX工具的加持下，其他深度学习计算框架也可以使用TensorBoard工具进行可视化操作了。在PyTorch原生的可视化工具不够友好的情况下，我们更推荐使用TensorBoardX来实现PyTorch的建模可视化。

根据此前菜菜老师介绍的PyTorch整体框架图，虽然我们可以在torch.utils模块调用TensorBoard，但TensorBoard并不和PyTorch同步维护，要使用Tensorboard，需要单独安装。

![Alt text](image-23.png)

    值得一提的是，TensorBoard采用的是本地开启服务，本地记录结果，Web读取结果的模式，和本地生成结果，和在REPL环境中即时读取的模式中由很大区别

In [1]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset


In [13]:
from torch.utils.tensorboard import SummaryWriter

## 12.2.1 SummaryWriter类与TensorBoard的基本使用

在进行建模过程可视化展示的过程中，核心使用的类就是SummaryWriter类，通过该类的使用，我们可以记录关键运算关系，模型数值指标等，进而可以进行展示。

不过，考虑到真实生产环境中，大规模深度学习模型都是部署在服务器上运行的，我们无法像在本地IDE上一边输入数据一边绘图观测结果，我们只能将需要记录的关键结果记录下来并保存在某个文件里，然后再在本地服务器上开启某项服务读取该文件保存的信息。因此，整个过程会比在本地绘图稍微复杂，但确是真实生产环境中的主流操作。

使用SummaryWriter进行记录并读取结果的一般过程如下：

![Alt text](image-24.png)


    服务器远程开启服务，本地通过Web服务来读取操作，也是在执行分布式计算过程中的常规操作

* 实例化Summarywriter

In [15]:
writer = SummaryWriter(log_dir= 'test')

实例化的过程中需要确定保存路径，当保存在主目录文件内时，只需要输入文件夹名称即可。当文件夹不存在时，系统会自动创建一个。每次实例化一次SummaryWriter，就会在对应文件夹内生成一个文件。单个文件夹内允许存在多个文件，在实际读取过程中会一次性读取文件夹内全部文件。

In [16]:
writer.log_dir      # 通过log_dir查看summary对象记录文件的位置

'test'

    此外，在实例化SummaryWriter类的时候还有一个常用的关键数——comment，通过comment参数的设置，我们可以自由设置每个实例化类的过程中文件名的后缀。

* 记录数据

In [17]:
for i in range(10):
    writer.add_scalar('mul', i**i, i)

此处我们使用了add_scalar方法进行数据记录，该方法用于记录一组标量，在实际记录过程中，通过迭代的方式逐个增加。其中，第一个参数代表该组记录的名称（也就是生成图像的名称），第二个参数代表y值，第三个参数代表x值。完成记录后，即可读取文件查看记录结果。除此之外，常用的还有add_graph方法用于记录深度学习建模流程。

* 启动服务读取文件

接下来，在本地启动服务，读取刚刚生成的记录文件。同样，我们可以在Jupyter中的命令行直接启动服务

F:

cd '/media/liu/0008E13F000D50BB/1/'

tensorboard --logdir='test'

    注意，如果不是通过Jupyter，而是通过cmd打开的命令行，如果要进入其他盘的主目录，例如F盘，则需要先输入F：指令，将当前操作主目录转移到F盘上。

命令行中的内容，其中tensorboard为指令关键词，--logdir参数输入记录文件夹地址（注意是文件夹地址而不是文件夹内的文件地址）。在默认情况下，系统会自动启动6006端口，可用于WEB调用服务，这种调用方法类似于Jupyter服务也是通过WEB调用的，jupyter端口默认是8888,需要注意的是，上述命令中，是先将当前操作目录转移至test文件目录下，然后再调用tensorboard命令时只需要在--logdir参数位输入文件名称即可。无论是在--logdir参数位输入完整的文件路径，还会四先将当前操作目录转移到对应目录下再直接输入文件名，都可以直接调用服务。

    命令行的操作时需要区分当前操作目录的，命令行中光标前面的内容即时当前操作目录，每次打开命令行时默认主目录都是当前用户的主目录，我们可以通过输入dir查看当前文件内容

* 进入6006端口页面

打开浏览器，在地址栏输入localhost:6006即可进入tensorboard展示页面。由于此外此前我们已经通过write记录了一组数据，我们可以看到其展示效果